In [8]:
from ipynb.fs.full.appendOnto import append_file, process_lines
from ipynb.fs.full.connectIndividuals import getNamespace, connectIndividuals, clear_serialize
from rdflib import Graph
from os import walk, stat

In [9]:
def getFilenames(input_path, extension=''):
    return [f for f in next(walk(input_path), (None, None, []))[2] if f.endswith('.ttl')]  # [] if no file

In [ ]:
input_path = "./1_ontologiesInput/"
connect_output_DIR = "./2_connectOutput/"
merge_output_DIR = "./3_mergeOutput/"
input = getFilenames(input_path, '.ttl')

In [11]:
"""
Every files in {input_path} will be updated with a new class,
and an individual from said class.
The class is named "CyberNews", and the individual is named after the file's name.
Every other individuals in the ontology will have object property of "relatedTo" to said individual. 
"""

connected = []
news_class = "CybersecurityNewsArticle"

for ind, file in enumerate(input):
    if stat(input_path + file).st_size == 0: # skip empty files
        continue
    g = Graph()
    process_lines(f"{input_path}{file}", f"{input_path}{file}")
    g.parse(f"{input_path}{file}", format="ttl")

    prefix, uri = getNamespace(g)
    news_name = file.split('.')[0]  # Get filename, exclude extension
    connectIndividuals(g, prefix, uri, news_name, news_class)   

    OutputDes = f"{connect_output_DIR}{news_name}_connected.ttl"  # Get new files' names
    connected.append(OutputDes)
    clear_serialize(g, OutputDes)

In [12]:
merged_file = f"{merge_output_DIR}UnifiedOntology.ttl"

In [13]:
for ind, connected_file in enumerate(connected):
    append_file(source_file=connected_file, target_file=merged_file)

In [14]:
g = Graph()
g.parse(merged_file, format="ttl")
clear_serialize(g, merged_file)